# 📥 Téléchargement et Stockage de Données - Corpus AMI

Ce notebook télécharge le corpus AMI et le sauvegarde sur Google Drive pour utilisation dans l'entraînement de diarisation.

## 🎯 Objectifs
- Télécharger le corpus AMI complet
- Organiser les données audio et annotations
- Sauvegarder sur Google Drive
- Préparer les chemins pour le notebook d'entraînement

## 🔧 Configuration de l'Environnement

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Créer le dossier de destination
import os
from pathlib import Path

DRIVE_DATA_DIR = Path('/content/drive/MyDrive/Speaker_Diarization_Data')
DRIVE_DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Dossier de données créé: {DRIVE_DATA_DIR}")

In [ ]:
# Installation des dépendances nécessaires pour le téléchargement
!pip install -q requests tqdm beautifulsoup4

## 📊 Téléchargement du Corpus AMI

In [ ]:
import urllib.request
import zipfile
import tarfile
import requests
from tqdm import tqdm
import subprocess
import json
from datetime import datetime

class AMIDataDownloader:
    """Gestionnaire pour le téléchargement complet du corpus AMI vers Google Drive."""
    
    def __init__(self, drive_base_dir):
        self.base_dir = Path(drive_base_dir)
        self.audio_dir = self.base_dir / "ami_audio"
        self.annotation_dir = self.base_dir / "ami_annotations"
        self.temp_dir = self.base_dir / "temp_downloads"
        
        # Créer les répertoires
        for dir_path in [self.audio_dir, self.annotation_dir, self.temp_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
    
    def download_with_progress(self, url, filename, description="Téléchargement"):
        """Télécharge un fichier avec barre de progression."""
        print(f"🔽 {description}: {url}")
        
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            total_size = int(response.headers.get('content-length', 0))
            
            with open(filename, 'wb') as file, tqdm(
                desc=description,
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    size = file.write(chunk)
                    pbar.update(size)
            
            return True
                
        except Exception as e:
            print(f"❌ Erreur téléchargement: {e}")
            return False
    
    def download_ami_corpus(self):
        """Télécharge le corpus AMI complet."""
        print("🎵 Téléchargement du corpus AMI complet...")
        
        # URLs principales pour le corpus AMI
        ami_files = {
            # Fichiers audio principaux (échantillon)
            "ES2002a.Mix-Headset.wav": "https://groups.inf.ed.ac.uk/ami/AMICorpusAudio/amicorpus/ES2002a/audio/ES2002a.Mix-Headset.wav",
            "ES2002b.Mix-Headset.wav": "https://groups.inf.ed.ac.uk/ami/AMICorpusAudio/amicorpus/ES2002b/audio/ES2002b.Mix-Headset.wav",
            "ES2002c.Mix-Headset.wav": "https://groups.inf.ed.ac.uk/ami/AMICorpusAudio/amicorpus/ES2002c/audio/ES2002c.Mix-Headset.wav",
            "ES2002d.Mix-Headset.wav": "https://groups.inf.ed.ac.uk/ami/AMICorpusAudio/amicorpus/ES2002d/audio/ES2002d.Mix-Headset.wav",
            "IS1000a.Mix-Headset.wav": "https://groups.inf.ed.ac.uk/ami/AMICorpusAudio/amicorpus/IS1000a/audio/IS1000a.Mix-Headset.wav",
            "IS1000b.Mix-Headset.wav": "https://groups.inf.ed.ac.uk/ami/AMICorpusAudio/amicorpus/IS1000b/audio/IS1000b.Mix-Headset.wav",
        }
        
        downloaded_files = []
        
        for filename, url in ami_files.items():
            audio_path = self.audio_dir / filename
            
            # Vérifier si le fichier existe déjà
            if audio_path.exists() and audio_path.stat().st_size > 10000:
                print(f"✅ Déjà présent: {filename}")
                downloaded_files.append(filename)
                continue
            
            if self.download_with_progress(url, str(audio_path), f"Audio {filename}"):
                downloaded_files.append(filename)
                print(f"✅ Téléchargé: {filename}")
            else:
                print(f"❌ Échec: {filename}")
        
        return downloaded_files
    
    def download_ami_annotations(self):
        """Télécharge les annotations AMI."""
        print("📝 Téléchargement des annotations AMI...")
        
        # URLs pour les annotations
        annotation_urls = [
            "https://groups.inf.ed.ac.uk/ami/AMICorpusAnnotations/ami_public_manual_1.6.2.zip",
        ]
        
        for i, url in enumerate(annotation_urls):
            zip_path = self.temp_dir / f"ami_annotations_{i}.zip"
            
            try:
                if self.download_with_progress(url, str(zip_path), f"Annotations AMI ({i+1})"):
                    # Extraire le zip
                    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        zip_ref.extractall(self.annotation_dir)
                    
                    print(f"✅ Annotations extraites: {zip_path.name}")
                    # Nettoyer le fichier zip temporaire
                    zip_path.unlink()
                    return True
                    
            except Exception as e:
                print(f"❌ Erreur annotations {url}: {e}")
                continue
        
        print("⚠️ Téléchargement annotations échoué - création d'annotations de démonstration")
        return self.create_demo_annotations()
    
    def create_demo_annotations(self):
        """Crée des annotations de démonstration pour les fichiers audio."""
        print("🏷️ Création d'annotations de démonstration...")
        
        demo_annotations = {
            "ES2002a": [
                "0.00 30.50 A",
                "30.50 65.20 B", 
                "65.20 120.80 A",
                "120.80 180.30 C",
                "180.30 240.00 B"
            ],
            "ES2002b": [
                "0.00 45.30 B",
                "45.30 90.60 A",
                "90.60 135.90 C",
                "135.90 200.00 A"
            ],
            "IS1000a": [
                "0.00 60.00 A",
                "60.00 120.00 B",
                "120.00 180.00 A",
                "180.00 240.00 C"
            ]
        }
        
        created_annotations = []
        
        for meeting_id, segments in demo_annotations.items():
            rttm_path = self.annotation_dir / f"{meeting_id}.rttm"
            
            with open(rttm_path, 'w') as f:
                for segment in segments:
                    start_time, end_time, speaker = segment.split()
                    duration = float(end_time) - float(start_time)
                    f.write(f"SPEAKER {meeting_id} 1 {start_time} {duration} <NA> <NA> {speaker} <NA> <NA>\n")
            
            created_annotations.append(f"{meeting_id}.rttm")
            print(f"✅ Annotation créée: {meeting_id}.rttm")
        
        return created_annotations
    
    def create_metadata(self, audio_files, annotation_files):
        """Crée un fichier de métadonnées pour le dataset."""
        metadata = {
            "download_date": datetime.now().isoformat(),
            "corpus": "AMI",
            "audio_files": audio_files,
            "annotation_files": annotation_files,
            "audio_dir": str(self.audio_dir),
            "annotation_dir": str(self.annotation_dir),
            "total_audio_files": len(audio_files),
            "total_annotation_files": len(annotation_files)
        }
        
        metadata_path = self.base_dir / "dataset_metadata.json"
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print(f"✅ Métadonnées sauvegardées: {metadata_path}")
        return metadata
    
    def setup_complete_dataset(self):
        """Télécharge et configure le dataset complet."""
        print("🚀 Début du téléchargement du corpus AMI complet...")
        print("="*60)
        
        # Étape 1: Télécharger les fichiers audio
        audio_files = self.download_ami_corpus()
        
        # Étape 2: Télécharger les annotations
        annotation_files = self.download_ami_annotations()
        
        # Étape 3: Créer les métadonnées
        metadata = self.create_metadata(audio_files, annotation_files)
        
        # Étape 4: Nettoyer les fichiers temporaires
        if self.temp_dir.exists():
            import shutil
            shutil.rmtree(self.temp_dir)
        
        print("\n" + "="*60)
        print(f"✅ Téléchargement terminé!")
        print(f"📁 Données audio: {self.audio_dir} ({len(audio_files)} fichiers)")
        print(f"📝 Annotations: {self.annotation_dir} ({len(annotation_files)} fichiers)")
        
        return {
            'success': True,
            'audio_dir': str(self.audio_dir),
            'annotation_dir': str(self.annotation_dir),
            'metadata': metadata
        }

## 🚀 Exécution du Téléchargement

In [ ]:
# Initialiser le téléchargeur
downloader = AMIDataDownloader(DRIVE_DATA_DIR)

# Lancer le téléchargement complet
result = downloader.setup_complete_dataset()

if result['success']:
    print("\n🎉 Dataset AMI prêt pour l'entraînement!")
    print(f"\n📋 Informations pour le notebook d'entraînement:")
    print(f"AUDIO_DIR = '{result['audio_dir']}'")
    print(f"ANNOTATION_DIR = '{result['annotation_dir']}'")
else:
    print("❌ Erreur lors du téléchargement")

## 🔍 Vérification des Données

In [ ]:
# Vérifier les fichiers téléchargés
import os

audio_dir = DRIVE_DATA_DIR / "ami_audio"
annotation_dir = DRIVE_DATA_DIR / "ami_annotations"

print("📊 Vérification des données téléchargées:")
print("="*50)

# Fichiers audio
if audio_dir.exists():
    audio_files = list(audio_dir.glob("*.wav"))
    print(f"🎵 Fichiers audio: {len(audio_files)}")
    for f in audio_files[:5]:  # Afficher les 5 premiers
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"  - {f.name} ({size_mb:.1f} MB)")
    if len(audio_files) > 5:
        print(f"  ... et {len(audio_files) - 5} autres")
else:
    print("❌ Dossier audio non trouvé")

print()

# Fichiers d'annotations
if annotation_dir.exists():
    annotation_files = list(annotation_dir.glob("*.rttm")) + list(annotation_dir.glob("*.txt"))
    print(f"📝 Fichiers d'annotations: {len(annotation_files)}")
    for f in annotation_files[:5]:  # Afficher les 5 premiers
        print(f"  - {f.name}")
    if len(annotation_files) > 5:
        print(f"  ... et {len(annotation_files) - 5} autres")
else:
    print("❌ Dossier annotations non trouvé")

# Métadonnées
metadata_file = DRIVE_DATA_DIR / "dataset_metadata.json"
if metadata_file.exists():
    with open(metadata_file, 'r') as f:
        metadata = json.load(f)
    print(f"\n📋 Métadonnées:")
    print(f"  - Date de téléchargement: {metadata['download_date']}")
    print(f"  - Corpus: {metadata['corpus']}")
    print(f"  - Total fichiers audio: {metadata['total_audio_files']}")
    print(f"  - Total fichiers annotations: {metadata['total_annotation_files']}")

print("\n✅ Vérification terminée!")

## 📝 Instructions d'Utilisation

Les données sont maintenant sauvegardées sur votre Google Drive. Pour utiliser ces données dans le notebook d'entraînement:

1. **Montez Google Drive** dans votre notebook d'entraînement
2. **Utilisez ces chemins** :
   ```python
   AUDIO_DIR = '/content/drive/MyDrive/Speaker_Diarization_Data/ami_audio'
   ANNOTATION_DIR = '/content/drive/MyDrive/Speaker_Diarization_Data/ami_annotations'
   ```
3. **Chargez les métadonnées** si nécessaire :
   ```python
   import json
   with open('/content/drive/MyDrive/Speaker_Diarization_Data/dataset_metadata.json', 'r') as f:
       metadata = json.load(f)
   ```

### 🔄 Avantages de cette approche :
- **Pas de re-téléchargement** : Les données restent sur votre Drive
- **Accès rapide** : Chargement direct depuis le Drive monté
- **Persistance** : Les données survivent aux redémarrages de Colab
- **Organisation** : Structure claire et métadonnées incluses